<a href="https://colab.research.google.com/github/Maynex69/Gestion-de-ingresos-hospitalarios/blob/main/Creaci%C3%B3n_de_nuevo_dataset_con_datos_m%C3%A1s_precisos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd
import numpy as np

from google.colab import files

# Cargar el dataset
df = pd.read_csv('/content/dataset.csv', sep=',', header=0, index_col=None)

# Contar el número de veces que aparece cada condición médica
condition_counts = df['Medical Condition'].value_counts()

# 1. Definir la distribución deseada para cada condición médica
desired_distribution = {
    'Diabetes': 0.21,
    'Arthritis': 0.16,
    'Obesity': 0.2,
    'Hypertension': 0.21,
    'Cancer': 0.1,
    'Asthma': 0.12
}

# 2. Contar el número total de pacientes
total_patients = len(df)

# 3. Calcular el número deseado de pacientes por condición
desired_counts = {cond: int(total_patients * perc) for cond, perc in desired_distribution.items()}

# 4. Crear un nuevo DataFrame con la redistribución deseada
new_df = pd.DataFrame()

for condition, desired_count in desired_counts.items():
    # Filtrar pacientes con la condición actual
    condition_df = df[df['Medical Condition'] == condition]

    # Si hay suficientes pacientes para la condición, muestrea o selecciona el número deseado
    if len(condition_df) > desired_count:
        condition_df = condition_df.sample(desired_count, random_state=42)  # Muestreo aleatorio
    else:
        # Si no hay suficientes pacientes, usa todos los disponibles
        condition_df = condition_df

    # Añadir al nuevo DataFrame
    new_df = pd.concat([new_df, condition_df], ignore_index=True)

# 5. Definir las edades y los porcentajes para cada condición
conditions = {
    'Diabetes': (60, 10, 0.8),
    'Arthritis': (75, 15, 0.9),
    'Obesity': (30, 25, 0.7),
    'Hypertension': (55, 15, 0.85),
    'Cancer': (65, 10, 0.95),
    'Asthma': (20, 15, 0.9)
}

for condition, (mean_age, std_dev, replace_percentage) in conditions.items():
    # Filtrar pacientes con la condición actual
    condition_df = new_df[new_df['Medical Condition'] == condition]

    # Calcular el número de pacientes a reemplazar
    num_to_replace = int(len(condition_df) * replace_percentage)

    # Seleccionar aleatoriamente pacientes para reemplazar
    indices_to_replace = np.random.choice(condition_df.index, num_to_replace, replace=False)

    # Generar edades basadas en una distribución normal
    new_ages = np.random.normal(mean_age, std_dev, num_to_replace).astype(int)

    # Asegurarte de que las edades estén dentro del rango de 5 a 100 años
    new_ages = np.clip(new_ages, 5, 100)

    # Aplicar los nuevos valores de edad
    new_df.loc[indices_to_replace, 'Age'] = new_ages

# 1. Definir medicamentos más realistas, su frecuencia y márgenes de edad para cada condición
medications = {
    'Diabetes': {
        'meds': (['Metformin', 'Insulin', 'Glipizide'], [0.2, 0.7, 0.1]),
        'age_ranges': [(5, 40), (41, 60), (61, 100)]  # Rangos de edad
    },
    'Arthritis': {
        'meds': (['Ibuprofen', 'Diclofenac', 'Methylprednisolone'], [0.5, 0.3, 0.2]),
        'age_ranges': [(31, 60), (5, 30), (61, 100)]
    },
    'Obesity': {
        'meds': (['Semaglutide', 'Phentermine', 'Lorcaserin'], [0.5, 0.3, 0.2]),
        'age_ranges': [(5, 40), (41, 60), (61, 100)]
    },
    'Hypertension': {
        'meds': (['Captopril', 'Nifedipine', 'Nitroglycerine'], [0.45, 0.35, 0.2]),
        'age_ranges': [(20, 50), (51, 70), (71, 100)]
    },
    'Cancer': {
        'meds': (['Paclitaxel', 'Cisplatin'], [0.6, 0.4]),
        'age_ranges': [(5, 40), (41, 100)]
    },
    'Asthma': {
        'meds': (['Albuterol', 'Fluticasone', 'Methylprednisolone'], [0.4, 0.5, 0.1]),
        'age_ranges': [(5, 18), (19, 35), (36, 100)]
    }
}

# 2. Reemplazar medicamentos en el DataFrame
for condition, info in medications.items():
    med_list, weights = info['meds']
    age_ranges = info['age_ranges']

    # Filtrar pacientes con la condición actual
    condition_df = new_df[new_df['Medical Condition'] == condition]

    for age_range in age_ranges:
        # Filtrar por rango de edad
        age_filtered_df = condition_df[(condition_df['Age'] >= age_range[0]) & (condition_df['Age'] <= age_range[1])]

        # Calcular el número de pacientes a reemplazar
        num_to_replace = int(len(age_filtered_df) * 1)  # Cambia el porcentaje según sea necesario

        if num_to_replace > 0:
            # Seleccionar aleatoriamente pacientes para reemplazar
            indices_to_replace = np.random.choice(age_filtered_df.index, num_to_replace, replace=False)

            # Generar medicamentos basados en las ponderaciones
            new_medications = np.random.choice(med_list, num_to_replace, p=weights)

            # Aplicar los nuevos valores de medicamento
            new_df.loc[indices_to_replace, 'Medication'] = new_medications

# Eliminar medicamentos que no están en la lista de medicamentos definidos
new_df = new_df[new_df['Medication'].isin(sum([info['meds'][0] for info in medications.values()], []))]


In [67]:

# Definir el porcentaje deseado por enfermedad
percentages = {
    'Diabetes': {'Male': 0.3, 'Female': 0.7},
    'Hypertension': {'Male': 0.6, 'Female': 0.4},
    'Arthritis': {'Male': 0.3, 'Female': 0.7},
    'Obesity': {'Male': 0.6, 'Female': 0.4},
    'Cancer': {'Male': 0.2, 'Female': 0.8},
    'Asthma': {'Male': 0.4, 'Female': 0.6}
}

# Crear una nueva columna para el género ajustado
adjusted_gender = []

for index, row in new_df.iterrows():
    condition = row['Medical Condition']
    if condition in percentages:
        male_percentage = percentages[condition]['Male']
        if np.random.rand() < male_percentage:
            adjusted_gender.append('Male')
        else:
            adjusted_gender.append('Female')
    else:
        adjusted_gender.append(row['Gender'])  # Mantener el género original si no se ajusta

# Asignar la nueva columna al DataFrame
new_df['Adjusted Gender'] = adjusted_gender

# Reemplazar 'OldColumn' por 'NewColumn'
new_df['Gender'] = new_df['Adjusted Gender']

# Si deseas eliminar la columna nueva después de reemplazar, puedes hacerlo así:
new_df.drop(columns=['Gender'], inplace=True)  # Elimina 'NewColumn' si ya no la necesitas

# Supongamos que has realizado tus manipulaciones en el DataFrame df
new_df.to_csv('mi_dataset.csv', index=False)

# Descargar el archivo
files.download('mi_dataset.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>